In [ ]:
#####################################
# Save using tf saver
# Only one default 'checkpoint' file is created per folder
# .meta file contains the information about the graph, i.e. how different vars are interacting
# .ckpt.data contains mapping of vars to their tensor values
# .ckpt.index gives indexing for lookup
#####################################

In [ ]:
# difference btw name_scope and variable_scope
# https://stackoverflow.com/questions/35919020/whats-the-difference-of-name-scope-and-a-variable-scope-in-tensorflow
# When the scope is specified with tf.name_scope(), the scope name is not added as header to the underlying vars and moreover,...
  # ...such scope name cannot be used for collecting specific graph keys like in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=name_scope_name)

In [ ]:
import tensorflow as tf

In [ ]:
# Saving a graph vars and re-loading those vars under a new graph name
class some_class(object):
    def __init__(self):
        self.w1 = tf.get_variable("w1", shape=[1,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
some_graph = tf.Graph();
with some_graph.as_default():
    some_object = some_class();
with tf.Session(graph=some_graph) as sess:
    saver = tf.train.Saver();
    sess.run(tf.global_variables_initializer());
    print(sess.run([some_object.w1])[0]);
    saver.save(sess, './w1', global_step=100)

some_new_graph = tf.Graph();
with some_new_graph.as_default():
    some_new_object = some_class();
with tf.Session(graph=some_new_graph) as sess:
    new_saver = tf.train.Saver();
    new_saver.restore(sess,'./w1-100');
    w1_new = some_new_graph.get_tensor_by_name('w1:0');
    print(sess.run([w1_new])[0]);

In [ ]:
# While reloading vars, the tf names matter and not the names you used to refer them in python script!
some_graph = tf.Graph();
with some_graph.as_default():
    w1 = tf.get_variable("w1", shape=[1,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
with tf.Session(graph=some_graph) as sess:
    sess.run(w1.initializer)
    saver = tf.train.Saver();
    saver.save(sess, './w1', global_step=1200)
    print(sess.run([w1])[0])

some_new_graph = tf.Graph();
with some_new_graph.as_default():
    w2 = tf.get_variable("w1", shape=[1,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
with tf.Session(graph=some_new_graph) as sess:
    new_saver = tf.train.Saver();
    new_saver.restore(sess, './w1-1200')
    print(sess.run([w2])[0])

In [ ]:
# While reloading a specific var fraom a checkpoint, its shape must match. Else you'll get InvalidArgumentError
# InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [2,5] rhs shape= [1,5]

# Creating a graph and saving vars
class some_class(object):
    def __init__(self):
        self.w1 = tf.get_variable("w1", shape=[1,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
some_graph = tf.Graph();
with some_graph.as_default():
    some_object = some_class();
with tf.Session(graph=some_graph) as sess:
    saver = tf.train.Saver();
    sess.run(tf.global_variables_initializer());
    print(sess.run([some_object.w1])[0]);
    saver.save(sess, './w1', global_step=500)
# Creating another graph with same name but with different size
class some_class(object):
    def __init__(self):
        self.w1 = tf.get_variable("w1", shape=[2,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
some_new_graph = tf.Graph();
with some_new_graph.as_default():
    some_new_object = some_class();
with tf.Session(graph=some_new_graph) as sess:
    new_saver = tf.train.Saver();
    new_saver.restore(sess,'./w1-500');
    w1_new = some_new_graph.get_tensor_by_name('w1:0');
    print(sess.run([w1_new])[0]);

In [ ]:
# While restoring, your new graph must have vars which are a subset of checkpoint vars_list. Else NotFoundError pops up
# NotFoundError: Key w2 not found in checkpoint

tf.reset_default_graph();
# Creating a graph and saving vars
class some_class(object):
    def __init__(self):
        self.w1 = tf.get_variable("w1", shape=[1,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
        self.w10 = tf.get_variable("w10", shape=[7,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
        self.w100 = tf.get_variable("w100", shape=[14,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
some_graph = tf.Graph();
with some_graph.as_default():
    some_object = some_class();
with tf.Session(graph=some_graph) as sess:
    saver = tf.train.Saver();
    sess.run(tf.global_variables_initializer());
    print(sess.run([some_object.w1])[0]);
    saver.save(sess, './first_graph', global_step=100)
# Creating another graph with some vars same as previous graph
class some_other_class(object):
    def __init__(self):
        # Run this...
        self.w1 = tf.get_variable("w1", shape=[1,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
        self.w1_0 = tf.get_variable("w10", shape=[7,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
        self.w2 = tf.get_variable("w2", shape=[3,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
some_new_graph = tf.Graph();
with some_new_graph.as_default():
    some_new_object = some_other_class();
with tf.Session(graph=some_new_graph) as sess:
    new_saver = tf.train.Saver();
    new_saver.restore(sess,'./first_graph-100');

In [ ]:
# Subset vars loading; loading vars under a specific name scope in tf graph; restroring specific variables based on name matching
# https://stackoverflow.com/questions/41621071/restore-subset-of-variables-in-tensorflow and https://www.tensorflow.org/api_docs/python/tf/train/Saver#__init__
# We'll make a collection of the requisite vars under a given name_scope and save them seperately

tf.reset_default_graph();
# Creating a graph and saving vars
class some_class(object):
    def __init__(self):
        self.base_scope = 'base_model_0';
        self.classifier_scope = 'classifier_model_0';
        with tf.variable_scope(self.base_scope, reuse=tf.AUTO_REUSE):
            with tf.variable_scope('layer_1'):
                w1 = tf.get_variable("w1", shape=[1,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
            with tf.variable_scope('layer_2'):
                w2 = tf.get_variable("w2", shape=[5,100], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
            self.w3 = tf.matmul(w1, w2, name='w3')
        with tf.variable_scope(self.classifier_scope, reuse=tf.AUTO_REUSE):
            w4 = tf.get_variable("w4", shape=[100,200], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
            self.w5 = tf.matmul(self.w3, w4, name='w5')
some_graph = tf.Graph();
with some_graph.as_default():
    some_object = some_class();
with tf.Session(graph=some_graph) as sess:
    sess.run(tf.global_variables_initializer());
    w3_out,w5_out = sess.run([some_object.w3,some_object.w5]);
    print(w3_out, w5_out);
    global_step = 550; # global_step = self.global_step; # Ideally, you update global steps in you optimizer code
    var_list_base = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=some_object.base_scope)
    print('Printing base list...')
    for var in var_list_base:
        print(var.name)
    saver_base = tf.train.Saver(var_list=var_list_base);
    saver_base.save(sess, './base_graph', global_step=global_step)
    var_list_full = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    saver_full = tf.train.Saver(var_list=var_list_full);
    saver_full.save(sess, './full_graph', global_step=global_step)
    print('Printing full list...')
    for var in var_list_full:
        print(var.name)
# Creating another graph with some vars same as previous graph but with DIFFERENT scope name
# returns NotFoundError: Key base_model_1/w1 not found in checkpoint
class some_other_class(object):
    def __init__(self):
        self.base_scope = 'base_model_1';
        self.classifier_scope = 'classifier_model_1';
        with tf.variable_scope(self.base_scope, reuse=tf.AUTO_REUSE):
            with tf.variable_scope('layer_1'):
                w1 = tf.get_variable("w1", shape=[1,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
            with tf.variable_scope('layer_2'):
                w2 = tf.get_variable("w2", shape=[5,100], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
            self.w3 = tf.matmul(w1, w2, name='w3')
        with tf.variable_scope(self.classifier_scope, reuse=tf.AUTO_REUSE):
            w4 = tf.get_variable("w4", shape=[100,33], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
            self.w5 = tf.matmul(self.w3, w4, name='w5')
some_new_graph = tf.Graph();
with some_new_graph.as_default():
    some_new_object = some_other_class();
with tf.Session(graph=some_new_graph) as sess:
    var_list_base = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=some_new_object.base_scope)
    new_saver_base = tf.train.Saver(var_list=var_list_base);
    new_saver_base.restore(sess,'./base_graph-550');
# Creating another graph with some vars same as previous graph but with SAME scope name
# returns no error unlike previous snippet!
class some_other_other_class(object):
    def __init__(self):
        self.base_scope = 'base_model_0';
        self.classifier_scope = 'classifier_model_2';
        with tf.variable_scope(self.base_scope, reuse=tf.AUTO_REUSE):
            with tf.variable_scope('layer_1'):
                w1 = tf.get_variable("w1", shape=[1,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
            with tf.variable_scope('layer_2'):
                w2 = tf.get_variable("w2", shape=[5,100], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
            self.w3 = tf.matmul(w1, w2, name='w3')
        with tf.variable_scope(self.classifier_scope, reuse=tf.AUTO_REUSE):
            w4 = tf.get_variable("w4", shape=[100,200], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
            self.w5 = tf.matmul(self.w3, w4, name='w5')
some_new_new_graph = tf.Graph();
with some_new_new_graph.as_default():
    some_new_new_object = some_other_other_class();
with tf.Session(graph=some_new_new_graph) as sess:
    var_list_base = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=some_new_new_object.base_scope)
    new_saver_base = tf.train.Saver(var_list=var_list_base);
    new_saver_base.restore(sess,'./full_graph-550'); # <--- Watch the name here and compare with next snippet
# Creating another graph with some vars same as previous graph but with SAME scope name
# returns no error unlike previous snippet!
class some_other_other_class(object):
    def __init__(self):
        self.base_scope = 'base_model_0';
        self.classifier_scope = 'classifier_model_2';
        with tf.variable_scope(self.base_scope, reuse=tf.AUTO_REUSE):
            with tf.variable_scope('layer_1'):
                w1 = tf.get_variable("w1", shape=[1,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
            with tf.variable_scope('layer_2'):
                w2 = tf.get_variable("w2", shape=[5,100], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
            self.w3 = tf.matmul(w1, w2, name='w3')
        with tf.variable_scope(self.classifier_scope, reuse=tf.AUTO_REUSE):
            w4 = tf.get_variable("w4", shape=[100,200], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
            self.w5 = tf.matmul(self.w3, w4, name='w5')
some_new_new_graph = tf.Graph();
with some_new_new_graph.as_default():
    some_new_new_object = some_other_other_class();
with tf.Session(graph=some_new_new_graph) as sess:
    var_list_base = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=some_new_new_object.base_scope)
    new_saver_base = tf.train.Saver(var_list=var_list_base);
    new_saver_base.restore(sess,'./full_graph-550');  # <--- Changes made here
    w3_out = sess.run([some_new_new_object.w3]);
    print(w3_out);
# This means, while restoring, just mark the appropriate variable_scope that has to be restored by your saver.
# While saving, you can save, as usual, the entire graph.